In [1]:
import fastprogress
import pandas as pd
from fastai.vision.all import *
from fastai.learner import *
import os
from pathlib import Path
from IPython.display import clear_output

# Setup

In [2]:
df = pd.read_csv("../data/test_labels.csv")
df["FILE_PATH"] = df.apply(lambda row: f"""../data/test_features/{row["FILE_NAME"]}""", axis=1)

In [3]:
def get_label_from_df(s):
    return ""

learner = load_learner(Path()/"models/model-stage11_.pkl",cpu=False)

# Predict on validation dataset

In [4]:
def predict(l, fp):
    prediction = l.predict(Path(fp))
    item_idx = prediction[1]
    return prediction[0], prediction[2][item_idx]

In [ ]:
df_predict = df.copy(deep=True)
df_predict = df_predict[["FILE_PATH","EXPECTED_CLASS"]]
df_predict["PREDICTED_CLASS"], df_predict["CONFIDENCE"] = zip(*df_predict["FILE_PATH"].apply(lambda fp: predict(learner, fp)))
clear_output()

# Compute Metrics

In [ ]:
df_predict["SUCCESS"] = (df_predict["EXPECTED_CLASS"]==df_predict["PREDICTED_CLASS"])
total_samples = len(df_predict)
success = len(df_predict[df_predict["SUCCESS"]==True])
accuracy = success / total_samples
print(f"Accuracy: {accuracy}")
print(f"Test sample size: {total_samples} images")

## Failed predictions

In [ ]:
df_predict[df_predict["SUCCESS"]==False]

## Demo

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

random_state = 42
expected_labels = df_predict["EXPECTED_CLASS"].unique().tolist()
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(40, 40))
for expected, ax in zip(expected_labels, axes.flat):
    random_sample = df_predict[df_predict["EXPECTED_CLASS"]==expected].sample(1)
    sample_path=random_sample["FILE_PATH"].values[0]
    predicted=random_sample["PREDICTED_CLASS"].values[0]
    confidence=random_sample["CONFIDENCE"].values[0].item()
    img = mpimg.imread(sample_path)
    ax.imshow(img)
    ax.set_title(f"Expected : {expected} | Predicted : {predicted} | CONFIDENCE: {confidence}", fontsize="18")